# COMS BC 3159 - S23
# PS3: Trajectory Optimization
* Due: 11:59pm, Friday March 10th

# Written Assignment (14 points)

Please go to the PDF in the written folder for the written instructions.

# Coding Assignment (26 points)
**You must submit to gradescope to get credit for the assignment --- make sure you submit the required files before the deadline!**

**NOTE: You need to install some python pacakges `pip install -r requirements.txt`**

**Files in this assignment:**
`pendulum.py`   Defines a series of (very useful) helper functions related to the robot (the pendulum) we are working with.
                You will need to finish some of the implementation here (and thus submit to Gradescope for Autograding).

`util.py`       The main file you will edit and implement helper functions (and thus submit to Gradescope for Autograding).

`trajopt.py`    Defines the high level trajectory optimzation functions that call the helper functions in `util` and `pendulum` in order to solve trajopt problems. You DO NOT need to submit this file.

`runPend.py`    Sets up some constants and calls the trajopt functions to run and graphically display the solve. You DO NOT need to submit this file.
                
                Usage is: -MAIN_SOLVER (-INNER_SOLVER) where
                    * MAIN_SOLVER  = [iLQR, SQP]
                    * INNER_SOLVER = [CG, INV]

In [ ]:
!pip3 install "pygame>2.1"

In [ ]:
!pip3 install "numpy>1.23" "scipy>1.9"


## Taylor Approximations of Dynamics and Cost Functions (7 Points)

For this part of the problem we'll be working in the `pendulum.py` file and we'll be implementing a few dynamics and cost helper functions that we'll use in the rest of the assignemnt.

Functions to update and their points:
* `next_state` 2 points
* `next_state_gradient` 2 points
* `cost_value` 1 point
* `cost_gradient` 1 point
* `cost_hessian` 1 point

We'll begin by converting the pendulum physics functions which compute acceleration (`dynamics`) and its gradient (`dynamics_gradient`) into `next_state` and `next_state_gradient` functions through the use of Euler integration. Recall from class that Euler integation adds a small amount of acceleration and velocity to the original position and velocity:
```
`[q', qd'] = [q, qd] + dt * [qd, qdd]`
```
Also recall that the dynamics derivative matricies are:
```
             A = [[dq'/dq, dq'/dqd],    and B = [[dq/du],
                  [dqd'/dq, dqd'/dqd]]           [dqd/du]]
```

Next we'll move onto the cost functions. Here we need to implement the cost value, gradient, and hessian. For this problem we are going to use the "standard simple quadratic cost":
```
0.5(x-xg)^TQ(x-xg) + 0.5u^TRu
```
As that outputs a scalar value the cost gradient and hessian therefore are of the form:
```
g = [dcost/dq, dcost/dqd, dcost/du]
H = [[d^2cost/dq^2,   d^2cost/dq dqd, d^2cost/dq du], 
     [d^2cost/dqd dq, dcost/dqd^2,    d^2cost/dqd du],
     [d^2cost/du dq,  dcost/du dqd,   d^2cost/du^2]]
```

Note that most things are of type [`np.array`](https://numpy.org/doc/stable/user/quickstart.html) which hopefully should simplify your linear algebra and there are a few more hints left in the code comments! Also, make sure to take into account both the states and the controls!

In [ ]:
from math import sin, cos, pi
import numpy as np
import copy

class Pendulum():
    def __init__(self, timestep = 0.15, gravity = 9.81, damping = 0.01, control_min = -5, control_max = 5, control_step = 0.25):
        self.timestep = timestep
        self.gravity = gravity
        self.damping = damping
        self.control_min = control_min
        self.control_max = control_max
        self.control_step = control_step
        self.Q = None
        self.R = None
        self.QF = None
        self.goal = None

    ######################
    #                    #
    # High level helpers #
    #                    #
    ######################

    # return the state and control size
    def get_state_size(self):
        return 2
    def get_control_size(self):
        return 1

    # clamp the input n to the range [smallest, largest]
    def clamp(self, n, smallest, largest): 
        return max(smallest, min(n, largest))

    # ensure angles are between -pi and pi
    def wrap_angles(self, angle):
        upper_bound = pi
        lower_bound = -pi
        while (angle < lower_bound):
            angle += 2*pi
        while (angle > upper_bound):
            angle -= 2*pi
        return np.array(angle)

    # compute the difference between two states making sure to wrap angles
    def state_delta(self, state1, state2):
        return np.array([self.wrap_angles(state1[0]-state2[0]), state1[1]-state2[1]])

    #####################################################
    #                                                   #
    #  Helper functions you need to fill out start here #
    #                                                   #
    #####################################################

    ######################
    #                    #
    #  Physics helpers   #
    #                    #
    ######################

    # apply the physics of the pendulum to compute acceleration
    def dynamics(self, x, u):
        position = x[0]
        velocity = x[1]
        qdd = u - self.gravity*sin(position) - self.damping*velocity
        return np.array(qdd)

    # compute the gradient of applying physics of the pendulum to compute acceleration
    def dynamics_gradient(self, x, u):
        position = x[0]
        velocity = x[1]
        # qdd = u - self.gravity*sin(position) - self.damping*velocity
        dqdd_dq = -self.gravity*cos(position)
        dqdd_dqd = -self.damping
        dqdd_du = 1
        return np.array([dqdd_dq, dqdd_dqd, dqdd_du])

    # compute the next state using euler integration
    def next_state(self, x, u):
        position = np.array(x[0])
        velocity = np.array(x[1])

        # first compute acceleration using dynamics
        # note: in this case we also clamp the input to control_min/max
        #       to avoid unrealistic steps (this can be safely ignored in the gradient)
        u_clamp = self.clamp(u, self.control_min, self.control_max)
        acceleration = self.dynamics(x,u_clamp)

        #
        # TODO
        #
        # Euler integrator [q', qd'] = [q, qd] + dt * [qd, qdd]
        # note: make sure to wrap_angles where appropriate
        #
        return np.array([0,0])

    # compute the gradient of the next state function using euler integration
    def next_state_gradient(self, x, u):
        position = x[0]
        velocity = x[1]
        A = np.array([[0, 0], [0, 0]])
        B = np.array([[0], [0]])

        #
        # TODO
        #
        # Euler integrator [q', qd'] = [q, qd] + dt * [qd, qdd]
        # Return the partial derivative matricies:
        #     A = [[dq'/dq, dq'/dqd],    and B = [[dq/du],
        #          [dqd'/dq, dqd'/dqd]]           [dqd/du]]
        #       
        #
        return A, B


    ######################
    #                    #
    #    Cost helpers    #
    #                    #
    ######################

    def set_Q(self, Q):
        self.Q = Q
    def set_R(self, R):
        self.R = R
    def set_QF(self, QF):
        self.QF = QF
    def set_goal(self, goal):
        self.goal = goal

    # compute the cost of the form 0.5(x-xg)^TQ(x-xg) + 0.5u^TRu
    # note that if u=None then it is the final state and there is no control (use QF)
    def cost_value(self, x, u = None):
        cost = 0
        # first compute the error between the current state and the goal
        delta = self.state_delta(x, self.goal)
        # then compute the cost for that error
        #
        # TODO
        #
        # hint: np.matmul may be useful!
        return 0

    # compute the gradient of the cost of the form 0.5(x-xg)^TQ(x-xg) + 0.5u^TRu
    # note that if u=None then it is the final state and there is no control (use QF)
    #
    # return the vector [dcost/dq, dcost/dqd, dcost/du]
    def cost_gradient(self, x, u = None):
        grad = []
        # first compute the error between the current state and the goal
        delta = self.state_delta(x, self.goal)
        # then compute the cost gradient with respect to the state
        #
        # TODO
        #
        if u is not None:
            return np.array([0,0,0])
        else:
            return np.array([0,0])

    # compute the hessian of the cost of the form 0.5(x-xg)^TQ(x-xg) + 0.5u^TRu
    # note that if u=None then it is the final state and there is no control (use QF)
    #
    # return the matrix [[d^2cost/dq^2,   d^2cost/dq dqd, d^2cost/dq du], 
    #                    [d^2cost/dqd dq, dcost/dqd^2,    d^2cost/dqd du],
    #                    [d^2cost/du dq,  dcost/du dqd,   d^2cost/du^2]]
    #
    # Hint: you may find the block_diag helper function helpful! (But you don't need to use it)
    #
    def cost_hessian(self, x, u = None):
        H = [[]]
        #
        # TODO
        #
        # hint: if u is None then you only need the hessian with respect to q, qd
        #
        if u is not None:
            return np.zeros((3,3))
        else:
            return np.zeros((2,2))

    def cost_gradient_hessian(self, x, u = None):
        return self.cost_hessian(x,u), self.cost_gradient(x,u)

    def block_diag(self, *arrs):
        """Create a block diagonal matrix from the provided arrays.

        Given the inputs `A`, `B` and `C`, the output will have these
        arrays arranged on the diagonal::

            [[A, 0, 0],
             [0, B, 0],
             [0, 0, C]]

        If all the input arrays are square, the output is known as a
        block diagonal matrix.

        Parameters
        ----------
        A, B, C, ... : array-like, up to 2D
            Input arrays.  A 1D array or array-like sequence with length n is
            treated as a 2D array with shape (1,n).

        Returns
        -------
        D : ndarray
            Array with `A`, `B`, `C`, ... on the diagonal.  `D` has the
            same dtype as `A`.

        References
        ----------
        .. [1] Wikipedia, "Block matrix",
               http://en.wikipedia.org/wiki/Block_diagonal_matrix

        Examples
        --------
        >>> A = [[1, 0],
        ...      [0, 1]]
        >>> B = [[3, 4, 5],
        ...      [6, 7, 8]]
        >>> C = [[7]]
        >>> print(block_diag(A, B, C))
        [[1 0 0 0 0 0]
         [0 1 0 0 0 0]
         [0 0 3 4 5 0]
         [0 0 6 7 8 0]
         [0 0 0 0 0 7]]
        >>> block_diag(1.0, [2, 3], [[4, 5], [6, 7]])
        array([[ 1.,  0.,  0.,  0.,  0.],
               [ 0.,  2.,  3.,  0.,  0.],
               [ 0.,  0.,  0.,  4.,  5.],
               [ 0.,  0.,  0.,  6.,  7.]])

        """
        if arrs == ():
            arrs = ([],)
        arrs = [np.atleast_2d(a) for a in arrs]

        bad_args = [k for k in range(len(arrs)) if arrs[k].ndim > 2]
        if bad_args:
            raise ValueError("arguments in the following positions have dimension "
                                "greater than 2: %s" % bad_args) 

        shapes = np.array([a.shape for a in arrs])
        out = np.zeros(np.sum(shapes, axis=0), dtype=arrs[0].dtype)

        r, c = 0, 0
        for i, (rr, cc) in enumerate(shapes):
            out[r:r + rr, c:c + cc] = arrs[i]
            r += rr
            c += cc
        return out

## Constructing and Solving KKT Systems (10 Points) and DDP Algorithm (9 points)
### Constructing and Solving KKT Systems (10 Points)

Now that we have built a full pendulum object with integrators and cost functions we can use that to build up and solve KKT systems using sucessive quadratic programming! All functions for this section (and the next DDP section) can be found in `util.py` and will be helper functions called by the functions in `trajopt.py`.

Functions to update and their points:
* `compute_total_cost` 1 point
* `compute_total_constraint_violation` 2 point
* `construct_KKT_system_blocks` 4 points
* `assemble_KTT_system` 1 point
*  `compute_merit_value` 1 point
* `sqp_line_search_criteria` 1 point

### Part 1: Helpers

We'll start by setting up some real helper functions that build directly off of the previous section.
* `compute_total_cost`: simply sums the cost across each timestep along the trajectory leveraging the cost functions your wrote in the previous section.
* `compute_total_constraint_violation`: similarly sums up the constraint error. In our case this is simply the dynamics constraints along the trajecotory as well as the initial state constraint (`x0 = xs`). Please use the [L1 norm](https://montjoile.medium.com/l0-norm-l1-norm-l2-norm-l-infinity-norm-7a7d18a4f40c) here (aka absolute sum).

### Part 2: Setting up the KKT System

`construct_KKT_system_blocks`: Now that we have those helper functions in place lets set up the KKT system. This is the big part of this section. Here we want to walk through the states and controls along the trajectory and build the full `G, g, C, c` matricies. Note that in this case `G` is the cost Hessian for the whole trajectory, `g` is the cost gradient, `C` is the constraint gradient, and `c` is the constraint value. Also note that we assume the state/control/lambda ordering discussed in class. That is:
```
[x_0, u_0, x_1, u_1 ... u_{N-1}, x_N, \lambda_0, \lamnbda_1 ... \lamnbda_N]^T
```
*Hint: what is the sparsity pattern you expect in those matricies?*

`assemble_KTT_system`: Now that we have set up the KKT system blocks we need to form it into the full KKT system!
```
[[G, C^T],  *  [[xu],       =  [[-g],   <--->   KKT * xul = kkt
 [C, 0  ]]      [\lambda]]      [c]]
```

### Part 3: Solving the KKT System

Now that we have the problem steup we need to use sequential quadratic programming to continously solve QPs. But, we need to be careful and apply a line search to make sure we are taking good steps toward the goal! Since this is a constrained optimization problem we need to balance primal optimiality (finding the minimum of the cost function) and dual feasibility (finding a solution that doens't violate the constraints).

`compute_merit_value`: we are going to go about doing this by forming what is called a "merit function." This function will be the function we line search over and it will be our guide of how to balance the optimality and feasibility of our solution. We will use the standard L1 merit function (Nocedal and Wright 15.4) where:
```
merit = cost_value + \mu * |constaint_value|
```

`sqp_line_search_criteria`: Finally, we'll evaluate the new trajectory as compared to the old trajectory and make sure we are improving in terms of making a sufficient decrease in both cost and constraint according to our merit function. 

*Hint: this one is simpler than you may think.*

### Part 4 - Explore
Now that you have a working SQP implementation run 
```
python3 runPend.py -SQP
```
What happens? Does the trajectory go all the way to the goal? Does the trajectory always look dynamically feasible? Check the terminal output to see the cost function and constraint violations at each iteration. If you change the parameters for the cost function by adjusting `Q` and `R` in that file what happens? Play around with it a little bit. Hopefully it will give you a little better intuition for what the math is doing! You can also switch between using the standard matrix inverse solution of the KKT system to using a conjugate gradient verison. Simply pass in `-CG` or `-INV` at the end of the line above and you will switch between using the two. Can you tell a difference?

### DDP Algorithm (9 points)

In this section, you'll be implementing the DDP algorithm (in particular the iLQR variant). Your goal will be to implement several functions in `util.py` to successfully optimize paths under a cost function. We'll be leveraging all of the work we have done before to help setup the problem and again the main loop code is in `trajopt.py`.

Functions that should be filled in for full credit in the `util.py` file are:
* `initialize_CTG`            - 1 point
* `backpropogate_CTG`         - 2 points
* `compute_du_K`              - 1 point
* `compute_new_CTG`           - 1 point
* `backward_pass_iterate`     - 1 points
* `compute_control_update`    - 1 point
* `rollout_trajectory`        - 1 point
* `ilqr_line_search_criteria` - 1 point

### Part 1 - The Backward Pass
We first compute the feedback control update along the backward pass using the math from the lecture slides (again this is not something you should memorize but it is helpful to understand how its working). Be careful with the sizes of various items and note that you will likely need to slice into items to get the subparts you are looking for.

`initialize_CTG`: We start by setting up the initial quadratic approximation of the cost-to-go (aka negative reward / value function) at the final state. Hint: what is the possible best cost you can have when you are at the last state? What is a quadratic approximation of that?

`backpropogate_CTG`: We now need to compute the CTG update at the previous state using our linear approximation of the dynamics and our quadratic approximation of the cost at that state and of the cost-to-go at the next state. To do this we will use the iLQR version of the math from the lecture slides. Also note that most things are numpy arrays and so the [np.matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) function will make your life a lot easier!

```
Qxx = lxx + fx^T V'xx fx
Quu = luu + fu^T V'xx fu
Qxu = lxu + fx^T V'xx fu
Qx = lx + fx^T V'x
Qu = lu + fu^T V'x
```

`compute_du_K`and `compute_new_CTG` Following that you'll need to use those outputs to construct K, du, and the new estimate of the cost-to-go Vxx, Vx again using the math from the lecture slides (see image below) and again most things are numpy arrays!

```
\delta u = = -Quu^{-1} (Qux \delta x + Qu) = K \delta x + du
Vx = Qx - Qxu du
Vxx = Qxx - Qxu K
```

* `backward_pass_iterate` Finally we'll put that all together and solve a full backwards pass iterate. That is, given `A, B, H, g` -- the cost and dynaics gradients and the cost Hessian --  as well as `Vxx_kp1, Vx_kp1` -- the next states CTG gradient and Hessian -- how can we output `duk, Kk, Vxx, Vx` -- the feedforward and feedback update to the controls and the currrent gradient and Hessian of the CTG.

*Hint: you mostly just need to call the previous few functions!*

### Part 3 - The Forward Pass
Finally, you'll want to compute the control updates in the forward pass based on the feedback controller you computed in the backward pass!

`compute_control_update`: Here we want to compute the change in a single control we want to apply at a single state. Remember that we have both a feedforward (du) term and a feedback (K) term. You'll also want to implement the line search version of the control update!

`rollout_trajectory`: Now you'll want to use that function to rollout a full new trajectory for a given line search iterate.

`ilqr_line_search_criteria`: Finally, you'll want to return a flag indicating whether that newly rolled out trajectory should be accepted or rejected. Remember in iLQR the constraints are implicit in the rollout, so we only need to worry about making sure we have improved in terms of optimality (cost).

*Hint: this one is simpler than you may think.*

### Part 4 - Explore
Now that you have a working DDP implementation run the `runPend.py` file. What happens? How does this work as compared to the SQP algorithm? Is it better? Worse? Slower? Faster? Does the trajectory go all the way to the goal? If you change the parameters for the cost function by adjusting `Q` and `R` in that file what happens? Play around with it a little bit. Hopefully it will give you a little better intuition for what the math is doing!

In [ ]:
import numpy as np
import copy

class Util:
    def __init__(self, robot_object):
        self.robot_object = robot_object

    #####################################################
    #                                                   #
    #  Helper functions you need to fill out start here #
    #                                                   #
    #####################################################

    #####################################################
    #                                                   #
    #    Initialization and Taylor Expansion Helpers    #
    #                                                   #
    #####################################################

    # Compute the total cost along the trajectory
    def compute_total_cost(self, x, u, nx, nu, N):
        #
        # TODO
        #
        # Hint: You may want to use the helper functions in the robot_object!
        #
        J = 0
        return J

    # compute the total constraint error
    def compute_total_constraint_violation(self, x, u, xs, nx, nu, N):
        # I'll start us off with th initial state constraint
        x_err = x[:,0] - xs
        const = np.linalg.norm(x_err, ord=1)
        # then we need to find all of the dynamics errors
        #
        # TODO
        #
        #
        return const

    # compute the dynamics and cost gradient and hessians
    def compute_approximation(self, xk, uk, nx, nu, k):
        Hk, gk = self.robot_object.cost_gradient_hessian(xk, uk)
        Ak, Bk = self.robot_object.next_state_gradient(xk, uk)
        return Ak, Bk, Hk, gk

    #####################################################
    #                                                   #
    #              KKT System Helpers                   #
    #                                                   #
    #####################################################
    
    # form the blocks of the KKT system matrix and vector
    def construct_KKT_system_blocks(self, x, u, xs, nx, nu, N):
        total_states = nx*N
        total_controls = nu*(N-1)
        total_states_controls = total_states + total_controls

        # placeholders of the right size for the blocks you need to compute
        G = np.zeros((total_states_controls, total_states_controls))
        g = np.zeros((total_states_controls, 1))
        C = np.zeros((total_states, total_states_controls)) # note for our setup constraints are equal to states
        c = np.zeros((total_states, 1))

        n = nx + nu

        # filling in the initial constraint gradient and value for you!
        C[0:nx, 0:nx] = np.eye(nx)
        c[0:nx, 0] = x[:,0] - xs

        # now compute all the cost gradients and hessians and the rest of the constraint gradients
        #
        # TODO
        #
        # Hint: Don't forget about the final cost gradient / hessian!
        #

        return G, g, C, c

    # form the main KKT system matrix and vector from its blocks
    def assemble_KTT_system(self, G, g, C, c, nx, nu, N):
        #
        # TODO
        #
        # Hint: h and v stack things up!
        #
        KKTMatrix = None
        KKTVector = None
        return KKTMatrix, KKTVector

    # compute the merit function value along a trajectory
    def compute_merit_value(self, x, u, xs, mu, nx, nu, N):
        #
        # TODO
        #
        # Hint: how are we balancing optimality and feasability?
        #  
        J = 0
        const = 0
        merit = 0
        return J, const, merit

    # line search flag for the SQP algorithm
    def sqp_line_search_criteria(self, x_new, u_new, J_new, const_new, merit_new, x, u, J, const, merit, nx, nu, N):
        #
        # TODO
        #
        # Hint: did things get better?
        #  
        return False

    #####################################################
    #                                                   #
    #            iLQR Backward Pass Helpers             #
    #                                                   #
    #####################################################

    # what is the quadratic approximation of the optimal solution
    # from the perspective of the last state?
    def initialize_CTG(self, x, nx, k):
        #
        # TODO
        #
        # Hint: You may want to use the helper functions in the robot_object!
        #
        Vxx = None
        Vx = None
        return Vxx, Vx

    # compute the quadratic estimate one state back along the trajectory
    def backpropogate_CTG(self, A, B, H, g, Vxx, Vx, nx, nu, k):
        #
        # TODO
        #
        # Hint: A = fx, B = fu, H = Jww, g = Jw where w is x or u!
        # Hint2: Everything is a np.array and np.matmul may be helpful!
        #
        Hxx = None
        Huu = None
        Hux = None
        gx = None
        gu = None
        return Hxx, Hux, Huu, gx, gu

    # given the above computation what is the feedback and feedforward update?
    def compute_kappa_K(self, Hxx, Hux, Huu, gx, gu, HuuInv, nx, nu, N):
        #
        # TODO
        #
        # Hint: Everything is a np.array and np.matmul may be helpful!
        #
        K = None
        kappa = None
        return kappa, K

    # given all that above what is the new CTG estimate at this state?
    def compute_new_CTG(self, Hxx, Hux, Huu, gx, gu, HuuInv, kappa, K, nx, nu, N):
        #
        # TODO
        #
        # Hint: Everything is a np.array and np.matmul may be helpful!
        #
        Vxx = None
        Vx = None
        return Vxx, Vx

    # put most of the above backward pass functions together and get from
    # the inputs to return the current kappa, K, Vxx, Vx
    def backward_pass_iterate(self, A, B, H, g, Vxx_kp1, Vx_kp1, nx, nu, k):
        #
        # TODO
        #
        # Hint: you mostly just need to call the helper functions defined above!
        #       and np.linalg.inv may be helpful!
        #
        kappak = None
        Kk = None
        Vxx = None
        Vx = None
        return kappak, Kk, Vxx, Vx

    #####################################################
    #                                                   #
    #           iLQR Forward Pass Helpers               #
    #                                                   #
    #####################################################

    # return u_new based on the current x_new, k, kappa, alpha, and original x
    def compute_control_update(self, x, x_new, K, kappa, alpha, nx, nu, N):
        #
        # TODO
        #
        # Hint: You may want to use the helper functions in the robot_object!
        # Hint2: Everything is a np.array and np.matmul may be helpful!
        # Hint3: We are computing the UPDATE to the controls
        #
        delta = self.robot_object.state_delta(x_new, x)
        return 0

    # rollout the full trajectory to produce x_new, u_new based on
    # x, u (the original trajectory), as well as K, kappa, alpha (the updates)
    def rollout_trajectory(self, x, u, K, kappa, alpha, nx, nu, N):
        x_new = copy.deepcopy(x)
        u_new = copy.deepcopy(u)
        #
        # TODO
        #
        # Hint: You may want to use the helper functions in the robot_object and in util!
        #        
        return x_new, u_new

    # make sure things got better
    def ilqr_line_search_criteria(self, x_new, u_new, J_new, x, u, J, nx, nu, N):
        #
        # TODO
        #
        # Hint: did things get better?
        #  
        return False

## Main Solver and Runner Code
`trajopt.py`    Defines the high level trajectory optimzation functions that call the helper functions in `util` and `pendulum` in order to solve trajopt problems. You DO NOT need to submit this file.

`runPend.py`    Sets up some constants and calls the trajopt functions to run and graphically display the solve. You DO NOT need to submit this file.
                
                Usage is: -MAIN_SOLVER (-INNER_SOLVER) where
                    * MAIN_SOLVER  = [iLQR, SQP]
                    * INNER_SOLVER = [CG, INV]

In [ ]:
# trajopt.py
# This program runs the main algorithms built on top of your util functions
# (Do not modify, but please read)
#
# Brian Plancher - Spring 2023
# Adapted from code written by Rus Tedrake and Scott Kuindersma

import sys, math, pygame, copy
from pygame.locals import *
from time import sleep
import numpy as np
import scipy as sp
np.set_printoptions(precision=3) # so things print nicer

class Trajopt:
    def __init__(self, robot_object, start_node, goal_node, N, XMAX_MIN, YMAX_MIN, \
                       MAX_ITER = 100, EXIT_TOL = 1e-3, ALPHA_FACTOR = 0.5, ALPHA_MIN = 1e-4, MU = 5):
        self.robot_object = robot_object # the robot_object with physics and cost functions
        self.MAX_ITER = MAX_ITER         # total ddp loops to try
        self.EXIT_TOL = EXIT_TOL         # This is the convergence criterion. We will declare success when the trajectory
                                         # is updated by a norm of less that 1e-4. DO NOT MODIFY.
        self.N = N                       # Number of nodes in a trajectory
        self.start_node = start_node
        self.goal_node = goal_node
        self.util = Util(self.robot_object)
        self.XMAX_MIN = XMAX_MIN         # max for drawing locically
        self.YMAX_MIN = YMAX_MIN         # max for drawing locically
        self.canvas_max = 640            # max for drawing pixels
        # set global line search parameters
        self.alpha_factor = ALPHA_FACTOR # how much to reduce alpha by each deeper search
        self.alpha_min = ALPHA_MIN       # minimum alpha to try
        self.mu = MU                     # merit function weighting        

    def draw_circle(self, node, size, color):
        percent_x = (node[0] + self.XMAX_MIN) / 2 / self.XMAX_MIN
        percent_y = (node[1] + self.YMAX_MIN) / 2 / self.YMAX_MIN
    
        scaled_x = int(self.canvas_max*percent_x)
        scaled_y = int(self.canvas_max*percent_y)    

        pygame.draw.circle(self.screen, color, (scaled_x, scaled_y), size)

    def draw_line(self, node1, node2, color):
        percent_x1 = (node1[0] + self.XMAX_MIN) / 2 / self.XMAX_MIN
        percent_y1 = (node1[1] + self.YMAX_MIN) / 2 / self.YMAX_MIN
        percent_x2 = (node2[0] + self.XMAX_MIN) / 2 / self.XMAX_MIN
        percent_y2 = (node2[1] + self.YMAX_MIN) / 2 / self.YMAX_MIN
    
        scaled_x1 = int(self.canvas_max*percent_x1)
        scaled_y1 = int(self.canvas_max*percent_y1)
        scaled_x2 = int(self.canvas_max*percent_x2)
        scaled_y2 = int(self.canvas_max*percent_y2)

        # check if angle wrapping (and don't draw lines across the screen)
        if node1[0] - node2[0] > math.pi:
            pass
        elif node1[0] - node2[0] < -math.pi:
            pass
        else:
            pygame.draw.line(self.screen, color, (scaled_x2, scaled_y2), (scaled_x1, scaled_y1))

    def init_screen(self):
        # initialize and prepare screen
        pygame.init()
        self.screen = pygame.display.set_mode((self.canvas_max,self.canvas_max))
        pygame.display.set_caption('PS3 - RRT')
        black = 20, 20, 40
        blue = 0, 0, 255
        green = 0, 255, 0
        self.screen.fill(black)
        self.draw_circle(self.start_node, 5, blue)
        self.draw_circle(self.goal_node, 5, green)
        pygame.display.update()

    def draw_trajectory(self, x):
        white = 255, 240, 200
        red = 255, 0, 0
        self.draw_circle(x[:,0], 2, white)
        for k in range(1,self.N):
            self.draw_circle(x[:,k], 2, white)
            self.draw_line(x[:,k-1], x[:,k], red)
        pygame.display.update()
        sleep(0.4)

    def wait_to_exit(self, x, u, DISPLAY_MODE = True):
        # if in test mode return the path
        if not DISPLAY_MODE:
            return x, u, K
        # Else wait for the user to see the solution to exit
        else:
            while(1):
                for e in pygame.event.get():
                    if e.type == QUIT or (e.type == KEYUP and e.key == K_ESCAPE):
                        sys.exit("Leaving because you requested it.")


    def solve(self, x, u, N, DISPLAY_MODE = False, MAIN_SOLVER = '-iLQR', INNER_SOLVER = '-CG'):
        # start up the graphics
        self.init_screen()

        # get constants
        nx = self.robot_object.get_state_size()
        nu = self.robot_object.get_control_size()

        # solve
        error = False
        if MAIN_SOLVER == '-iLQR':
            self.iLQR(x, u, nx, nu, N, DISPLAY_MODE)
        elif MAIN_SOLVER == '-SQP':
            if INNER_SOLVER == '-CG':
                self.SQP(x, u, nx, nu, N, DISPLAY_MODE, 1)
            elif INNER_SOLVER == '-INV':
                self.SQP(x, u, nx, nu, N, DISPLAY_MODE, 0)
            else:
                print("[!] ERROR: Invalid Inner Solver: ", INNER_SOLVER)
                error = True
        else:
            print("[!] ERROR: Invalid Solver: ", MAIN_SOLVER)
            error = True
        return error


    def SQP(self, x, u, nx, nu, N, DISPLAY_MODE = False, INNER_SOLVER = 0):
        # compute initial merit function value
        xs = copy.deepcopy(x[:,0])
        J, const, merit = self.util.compute_merit_value(x, u, xs, self.mu, nx, nu, N)
        J_prev = J
        if DISPLAY_MODE:
            print("Initial Cost: ", J)
            print("Initial Constraint Violation: ", const)
            print("Initial Merit Function: ", merit)

        # start the main loop
        iteration = 0
        failed = False
        while 1:

            # construct the KKT system blocks
            G, C, g, c = self.util.construct_KKT_system_blocks(x, u, xs, nx, nu, N)

            # assemble into full KKT system
            KKTMatrix, KKTVector = self.util.assemble_KTT_system(G, C, g, c, nx, nu, N)

            # solve the KKT system
            if INNER_SOLVER == 0: # backslash (standard matrix inverse / factorization)
                dxul_new = np.linalg.solve(KKTMatrix,KKTVector)[:,0]
            else: # conjugate gradient
                dxul_new = sp.sparse.linalg.cg(KKTMatrix,KKTVector)[0]

            # do line search to accept (or reject) the update
            alpha = 1
            while 1:
                # unravel the update
                x_new = copy.deepcopy(x)
                u_new = copy.deepcopy(u)
                for k in range(N-1):
                    x_new[:,k] += alpha*dxul_new[(nx+nu)*k:(nx+nu)*k + nx]
                    u_new[:,k] += alpha*dxul_new[(nx+nu)*k + nx:(nx+nu)*(k+1)]
                x_new[:,N-1] += alpha*dxul_new[(nx+nu)*(N-1):(nx+nu)*(N-1) + nx]

                # compute the cost, constraint violation, and merit function
                J_new, const_new, merit_new = self.util.compute_merit_value(x_new, u_new, xs, self.mu, nx, nu, N)

                # compute the line search criteria
                flag = self.util.sqp_line_search_criteria(x_new, u_new, J_new, const_new, merit_new, x, u, J, const, merit, nx, nu, N)

                if flag:
                    J_prev = J
                    x = x_new
                    u = u_new
                    J = J_new
                    const = const_new
                    merit = merit_new
                    if DISPLAY_MODE:
                        print("Iter[", iteration, "] Cost[", np.round(J,4), "], Constraint Violation[", np.round(const,4), "], ", \
                                                     "mu [", self.mu, "], Merit Function[", np.round(merit,4), "] and x final:")
                        print(x[:,N-1])
                        self.draw_trajectory(x)
                    break

                # failed try to continue the line search
                elif alpha > self.alpha_min:
                    alpha *= self.alpha_factor
                    if DISPLAY_MODE:
                        print("Deepening the line search")
                
                # failed line search
                else:
                    if DISPLAY_MODE:
                        print("Line search failed")
                    failed = True
                    break

            # Check for exit (or error)
            if failed: # need double break to get out of both loops here if line search fails
                break

            delta_J = J_prev - J
            if delta_J < self.EXIT_TOL:
                if DISPLAY_MODE:
                    print("Exiting for exit_tolerance")
                break
            
            if iteration == self.MAX_ITER - 1:
                if DISPLAY_MODE:
                    print("Exiting for max_iter")
                break
            else:
                iteration += 1

        if DISPLAY_MODE:
            print("Final Trajectory")
            print(x)
            print(u)

        self.wait_to_exit(x, u, DISPLAY_MODE)

    def iLQR(self, x, u, nx, nu, N, DISPLAY_MODE = False):
        
        # allocate memory for things we will compute
        kappa = np.zeros([nu,N-1])     # control updates
        K = np.zeros((nu,nx,N-1))   # feedback gains

        # compute initial cost
        J = self.util.compute_total_cost(x, u, nx, nu, N)
        J_prev = J
        if DISPLAY_MODE:
            print("Initial Cost: ", J)

        # start the main loop
        iteration = 0
        failed = False
        while 1:

            # Do backwards pass to compute new control update and feedback gains: kappa and K
            # start by initializing the cost to go
            Vxx, Vx = self.util.initialize_CTG(x[:,N-1], nx, N) 
            for k in range(N-2,-1,-1):
                # then compute the quadratic approximation at that point
                A, B, H, g = self.util.compute_approximation(x[:,k], u[:,k], nx, nu, k)

                # then compute the control update and new CTG estimates
                kappak, Kk, Vxx, Vx = self.util.backward_pass_iterate(A, B, H, g, Vxx, Vx, nx, nu, k)

                # save kappa and K for forward pass
                kappa[:,k] = kappak.tolist()
                K[:,:,k] = Kk.tolist()

            # Do forwards pass to compute new x, u, J (with line search)
            alpha = 1
            while 1:
                # rollout new trajectory
                x_new, u_new = self.util.rollout_trajectory(x, u, K, kappa, alpha, nx, nu, N)

                # compute new cost
                J_new = self.util.compute_total_cost(x_new, u_new, nx, nu, N)
                    
                # simple line search criteria
                flag = self.util.ilqr_line_search_criteria(x_new, u_new, J_new, x, u, J, nx, nu, N)

                if flag:
                    J_prev = J
                    x = x_new
                    u = u_new
                    J = J_new
                    if DISPLAY_MODE:
                        print("Iteration[", iteration, "] with cost[", round(J,4), "] and x final:")
                        print(x[:,N-1])
                        self.draw_trajectory(x)
                    break

                # failed try to continue the line search
                elif alpha > self.alpha_min:
                    alpha *= self.alpha_factor
                    if DISPLAY_MODE:
                        print("Deepening the line search")
                
                # failed line search
                else:
                    if DISPLAY_MODE:
                        print("Line search failed")
                    failed = True
                    break

            # Check for exit (or error)
            if failed: # need double break to get out of both loops here if line search fails
                break

            delta_J = J_prev - J
            if delta_J < self.EXIT_TOL:
                if DISPLAY_MODE:
                    print("Exiting for exit_tolerance")
                break
            
            if iteration == self.MAX_ITER - 1:
                if DISPLAY_MODE:
                    print("Exiting for max_iter")
                break
            else:
                iteration += 1

        if DISPLAY_MODE:
            print("Final Trajectory")
            print(x)
            print(u)

        self.wait_to_exit(x, u, DISPLAY_MODE)

In [ ]:
# tricking colab to run pygame per https://colab.research.google.com/drive/1xtiBrGeRHmXY3KSOixkZBf_rJIgBImJu?usp=sharing#scrollTo=7X42jJWlAuSl
import cv2
from google.colab.patches import cv2_imshow
from google.colab import output
import time 
import os, sys
# set SDL to use the dummy NULL video driver, 
#   so it doesn't need a windowing system.
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [ ]:
from math import pi
import numpy as np
import sys

XMAX_MIN = pi
YMAX_MIN = 10

X_START = [0,0]
X_GOAL = [pi,0]

# How did I get these numbers? A little intuition and a lot of guess and check.
QF = np.array([[30,0],[0,10]])
Q = np.array([[3,0],[0,1]])
R = np.array([0.08])

N = 32

pend = Pendulum()
pend.set_Q(Q)
pend.set_R(R)
pend.set_QF(QF)
pend.set_goal(X_GOAL)

x0 = np.zeros([pend.get_state_size(),N])
u0 = np.zeros([pend.get_control_size(),N-1])

trajopt_obj = Trajopt(pend, X_START, X_GOAL, N, XMAX_MIN, YMAX_MIN)

## Function Calls for Tests
Note that only the text to command line will display -- even though we tricked the video driver it doesn't pop up the pygame window (you'll need to run it locally for the fun visualizaiton). **So you'll need to kill each cell with the stop button!**

In [ ]:
trajopt_obj.solve(x0, u0, N, True, '-SQP', '-INV')

In [ ]:
trajopt_obj.solve(x0, u0, N, True, '-SQP', '-CG')

In [ ]:
trajopt_obj.solve(x0, u0, N, True, '-iLQR')